In [ ]:
! pip install mpu

     |████████████████████████████████| 69 kB 3.3 MB/s 


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline  
import mpu

In [ ]:
def find_routes_with_ten_readings(df, route_numbers, min_num_readings=10, verbose=False):
    routes = []
    
    for number in route_numbers:
        route_df = df[df['route_number'] == number]
        
        if len(route_df) >= min_num_readings:
            routes.append(route_df)
        elif verbose:
            print('Route: ', number, ' only has ', len(route_df), ' readings!')
    
    print('Found', len(routes), 'routes that have', min_num_readings, 'or more readings')
    
    return pd.concat(routes)

In [ ]:
def load_csv_as_df(file_name, sub_directories, column_numbers=None, column_names=None):
    '''
    Load any csv as a pandas dataframe. Provide the filename, the subdirectories, and columns to read(if desired).
    '''
    base_path = os.getcwd()
    full_path = base_path + sub_directories + file_name

    if column_numbers is not None:
        df = pd.read_csv(full_path, usecols=column_numbers)
    else:
        df = pd.read_csv(full_path)

    if column_names is not None:
        df.columns = column_names
    
    route_ids = df['route_number'].unique()
    return find_routes_with_ten_readings(df, route_ids, min_num_readings=10)

In [ ]:
north_to_west_routes_df = load_csv_as_df('north-to-west-routes-with-cells.csv', '/')
west_to_north_routes_df = load_csv_as_df('west-to-north-routes-with-cells.csv', '/')

Found 196 routes that have 10 or more readings
Found 283 routes that have 10 or more readings


In [ ]:
north_to_west_routes_df.head()

,taxi_id,time,longitude,latitude,occupancy_status,speed,route_number,route_start,route_end,cell,row,column
0,22391,2018-12-08 22:02:44,114.026115,22.610600,1,43,501872,True,False,12-20,12,20
1,22391,2018-12-08 22:03:04,114.023849,22.609200,1,50,501872,False,False,12-20,12,20
2,22391,2018-12-08 22:03:24,114.022003,22.607018,1,58,501872,False,False,12-20,12,20
3,22391,2018-12-08 22:03:44,114.023666,22.604549,1,58,501872,False,False,12-20,12,20
4,22391,2018-12-08 22:04:04,114.025284,22.602533,1,49,501872,False,False,12-20,12,20


In [ ]:
west_to_north_routes_df.head()

,taxi_id,time,longitude,latitude,occupancy_status,speed,route_number,route_start,route_end,cell,row,column
0,22262,2018-12-08 09:42:28,114.121231,22.547068,1,5,500264,True,False,10-22,10,22
1,22262,2018-12-08 09:42:58,114.123497,22.547533,1,27,500264,False,False,10-22,10,22
2,22262,2018-12-08 09:43:26,114.123619,22.547800,1,14,500264,False,False,10-22,10,22
3,22262,2018-12-08 09:43:29,114.123581,22.547783,1,0,500264,False,False,10-22,10,22
4,22262,2018-12-08 09:43:32,114.123535,22.547783,1,0,500264,False,False,10-22,10,22


In [ ]:
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date: pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [ ]:
def calculate_route_durations(df):
    route_durations = {}
    df['time'] = lookup(df['time'])
    route_ids = df['route_number'].unique()

    for route_id in route_ids:
        route_df = df[df['route_number'] == route_id]

        start_row = route_df[route_df['route_start'] == True]
        end_row = route_df[route_df['route_end'] == True]

        has_start_and_end = True
        if len(start_row) == 0:
            print('No start for route: ', route_id)
            has_start_and_end = False

        if len(end_row) == 0:
            print('No end for route: ', route_id)
            has_start_and_end = False

        if has_start_and_end:
            start_time = start_row['time'].iloc[0]
            end_time = end_row['time'].iloc[0]

            if end_time < start_time:
                print('End time earlier than start time for route number ', route_id)
                print()

            route_duration = end_time - start_time
            # print('route_duration ', route_duration)

            duration_in_seconds = route_duration.total_seconds()

            # print('Route ', route_id, ' duration in seconds ', duration_in_seconds)

            route_durations[route_id] = duration_in_seconds

    duration_df = pd.DataFrame(list(route_durations.items()), columns=['route_number', 'duration_in_seconds'])
    return duration_df

In [ ]:
north_to_west_duration_df = calculate_route_durations(north_to_west_routes_df)
north_to_west_duration_df.describe()

,route_number,duration_in_seconds
count,196.000000,196.000000
mean,591157.846939,1848.209184
std,52903.401810,678.761755
min,501872.000000,708.000000
25%,542577.500000,1317.250000
50%,596413.000000,1706.000000
75%,632940.000000,2208.000000
max,682586.000000,4369.000000


In [ ]:
west_to_north_duration_df = calculate_route_durations(west_to_north_routes_df)
west_to_north_duration_df.describe()

,route_number,duration_in_seconds
count,283.000000,283.000000
mean,592998.600707,1603.752650
std,52955.039419,484.280056
min,500264.000000,809.000000
25%,551146.000000,1288.500000
50%,594651.000000,1491.000000
75%,639403.000000,1836.000000
max,682626.000000,4215.000000


In [ ]:
def distance_between_gps(gps_one, gps_two):
    # mpu.haversine_distance((lat1, lon1), (lat2, lon2))
    km_distance = mpu.haversine_distance((gps_one[0], gps_one[1]), (gps_two[0], gps_two[1]))

    if km_distance < 0:
        print('got negative distance that\'s weak')
        km_distance *= -1

    return km_distance

In [ ]:
def calculate_route_distances(df):
    route_distances = {}
    df['time'] = lookup(df['time'])
    route_ids = df['route_number'].unique()

    for route_id in route_ids:
        route_df = df[df['route_number'] == route_id]
        route_df.sort_values('time')
        route_df.reset_index(drop=True)

        distance_sum = 0.0
        is_first_row = True
        
        for index, row in route_df.iterrows():
            if not is_first_row:
                last_row = route_df.loc[index - 1]
                last_lat = last_row['latitude']
                last_long = last_row['longitude']
                last_gps = (last_lat, last_long)

                current_lat = row['latitude']
                current_long = row['longitude']
                current_gps = (current_lat, current_long)

                distance_between_rows = distance_between_gps(last_gps, current_gps)
                distance_sum += distance_between_rows
            else:
                is_first_row = False

        route_distances[route_id] = distance_sum

    distance_df = pd.DataFrame(list(route_distances.items()), columns=['route_number', 'distance_in_km'])
    return distance_df

In [ ]:
north_to_west_distance_df = calculate_route_distances(north_to_west_routes_df)
north_to_west_distance_df.describe()

,route_number,distance_in_km
count,196.000000,196.000000
mean,591157.846939,15.953792
std,52903.401810,3.233245
min,501872.000000,9.997670
25%,542577.500000,13.532529
50%,596413.000000,16.004764
75%,632940.000000,17.588881
max,682586.000000,26.372868


In [ ]:
west_to_north_distance_df = calculate_route_distances(west_to_north_routes_df)
west_to_north_distance_df.describe()

,route_number,distance_in_km
count,283.000000,283.000000
mean,592998.600707,15.766159
std,52955.039419,3.139819
min,500264.000000,9.209701
25%,551146.000000,13.349983
50%,594651.000000,15.948325
75%,639403.000000,17.084236
max,682626.000000,29.376610


In [ ]:
nw_time_distance_df = pd.merge(north_to_west_duration_df, north_to_west_distance_df, on="route_number")
# nw_time_distance_df.to_csv('north_to_west_time_distance_df.csv', encoding='utf-8', index=False)
nw_time_distance_df.describe()

,route_number,duration_in_seconds,distance_in_km
count,196.000000,196.000000,196.000000
mean,591157.846939,1848.209184,15.953792
std,52903.401810,678.761755,3.233245
min,501872.000000,708.000000,9.997670
25%,542577.500000,1317.250000,13.532529
50%,596413.000000,1706.000000,16.004764
75%,632940.000000,2208.000000,17.588881
max,682586.000000,4369.000000,26.372868


In [ ]:
wn_time_distance_df = pd.merge(west_to_north_duration_df, west_to_north_distance_df, on="route_number")
# wn_time_distance_df.to_csv('west_to_north_time_distance_df.csv', encoding='utf-8', index=False)
wn_time_distance_df.describe()


,route_number,duration_in_seconds,distance_in_km
count,283.000000,283.000000,283.000000
mean,592998.600707,1603.752650,15.766159
std,52955.039419,484.280056,3.139819
min,500264.000000,809.000000,9.209701
25%,551146.000000,1288.500000,13.349983
50%,594651.000000,1491.000000,15.948325
75%,639403.000000,1836.000000,17.084236
max,682626.000000,4215.000000,29.376610


In [ ]:
def merge_distance_time_into_route_df(dt_df, df):
    route_dfs = []
    route_ids = df['route_number'].unique()

    for route_id in route_ids:
        route_df = df[df['route_number'] == route_id]
        distance_time_df = dt_df[dt_df['route_number'] == route_id]
        
        distance = distance_time_df['distance_in_km'].iloc[0]
        time = distance_time_df['duration_in_seconds'].iloc[0]
        
        route_df['distance_in_km'] = distance
        route_df['duration_in_seconds'] = time
        
        route_dfs.append(route_df)
    
    return pd.concat(route_dfs)

In [ ]:
def reduce_dataframe_by_col(df, col_name):
    row_dfs = []
    unique_values = df[col_name].unique()

    for val in unique_values:
        val_df = df[df[col_name] == val]

        row_dfs.append(val_df.iloc[[0]])
    
    return pd.concat(row_dfs)

In [ ]:
merged_north = merge_distance_time_into_route_df(nw_time_distance_df, north_to_west_routes_df)
merged_north = reduce_dataframe_by_col(merged_north, 'route_number')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
merged_west = merge_distance_time_into_route_df(wn_time_distance_df, west_to_north_routes_df)
merged_west = reduce_dataframe_by_col(merged_west, 'route_number')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
def find_fraud_by_tie_distance(df,avg_time,avg_distance):
  return df[(df["distance_in_km"]>=avg_distance) &(df["duration_in_seconds"]>=avg_time)]
  

In [ ]:
merged_west.describe()

,taxi_id,longitude,latitude,occupancy_status,speed,route_number,row,column,distance_in_km,duration_in_seconds
count,283.000000,283.000000,283.000000,283.0,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000
mean,30444.876325,114.109491,22.545812,1.0,9.936396,592998.600707,10.293286,21.805654,15.766159,1603.752650
std,4398.303968,0.011087,0.006813,0.0,13.719435,52955.039419,0.456075,0.396398,3.139819,484.280056
min,22262.000000,114.085648,22.531549,1.0,0.000000,500264.000000,10.000000,21.000000,9.209701,809.000000
25%,26799.500000,114.102314,22.541316,1.0,0.000000,551146.000000,10.000000,22.000000,13.349983,1288.500000
50%,31304.000000,114.111900,22.545450,1.0,4.000000,594651.000000,10.000000,22.000000,15.948325,1491.000000
75%,34408.500000,114.117058,22.551416,1.0,16.000000,639403.000000,11.000000,22.000000,17.084236,1836.000000
max,36918.000000,114.130280,22.559168,1.0,85.000000,682626.000000,11.000000,22.000000,29.376610,4215.000000


In [ ]:
north_fraud = find_fraud_by_tie_distance(merged_north, 1849, 16)
print(len(north_fraud))

55


In [ ]:
with open("North_to_West_Suspected_Fraud_By_Time_Distance.txt","w") as f:
  for item in north_fraud["route_number"].unique():
    f.write("%s\n" % item)

In [ ]:
west_fraud = find_fraud_by_tie_distance(merged_west, 1604, 16)
west_fraud

,taxi_id,time,longitude,latitude,occupancy_status,speed,route_number,route_start,route_end,cell,row,column,distance_in_km,duration_in_seconds
0,22262,2018-12-08 09:42:28,114.121231,22.547068,1,5,500264,True,False,10-22,10,22,18.625486,2671.0
192,22607,2018-12-08 11:10:49,114.117218,22.548086,1,9,504429,True,False,10-22,10,22,17.541450,2250.0
358,22736,2018-12-08 13:44:32,114.117401,22.544983,1,2,506123,True,False,10-22,10,22,16.276415,2284.0
683,22760,2018-12-08 14:22:43,114.116920,22.548134,1,14,506440,True,False,10-22,10,22,16.700748,1718.0
949,23225,2018-12-08 11:38:22,114.113831,22.544233,1,1,510811,True,False,10-22,10,22,16.111875,1644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20086,36550,2018-12-08 14:13:49,114.112717,22.533934,1,12,675044,True,False,10-22,10,22,17.569075,1942.0
20392,36670,2018-12-08 14:32:54,114.115417,22.535042,1,19,677982,True,False,10-22,10,22,22.609374,2040.0
20510,36755,2018-12-08 05:48:01,114.117119,22.538813,1,4,679657,True,False,10-22,10,22,16.112354,1673.0
20542,36780,2018-12-08 12:51:48,114.103600,22.545040,1,12,680209,True,False,10-22,10,22,25.508585,1807.0


In [ ]:
len(west_fraud)

78

In [ ]:
with open('West_to_North_Suspected_Fraud_By_Time_Distance.txt', 'w') as f:
    for item in west_fraud['route_number'].unique():
        f.write("%s\n" % item)